In [1]:
import pygame
import itertools
import random
import numpy as np

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
### Parameters ###

H = 900
B = 250
MAX_SPEED = 2 # unit?
FOLLOWING_TIME = 40
BUFFER = 20
SPAWN_PROBABILITY = 0.015
CAR_WIDTH = 10
CAR_LENGTH = (20,27)
TRUCK_LENGTH = (42,50)
TRUCK_PROBABILITY = 0.1
ACCELERATION = 0.02
DECELERATION = -0.02

In [3]:
class Car():
    def __init__(self, length, path):
        self.length = length
        self.sections = path.sections
        self.blockades = path.blockades
        self.i = 0
        self.done = False
        self.next_car = None
        self.find_next_car(0)
        self.distance = 0
        self.speed = MAX_SPEED
        self.acceleration = 0

    def section(self):
        return self.sections[self.i]
    
    def position(self):
        return self.section().position(self.distance)
    
    def angle(self):
        return self.section().angle(self.distance)
            
    def update_acceleration(self):
        self.acceleration = ACCELERATION
        next_car = self.next_car
        
        if self.i == 1:
            if any(b.last_car for b in self.blockades):
                space = self.sections[1].length - self.distance - self.length/2
                acceleration = -self.speed**2 / (2*space) if space else 0
                if acceleration > DECELERATION:
                    self.acceleration = acceleration
        if next_car:
            space = next_car.distance - self.distance - \
                    (self.length + next_car.length)/2 - BUFFER
            
            for i in range(self.i, len(self.sections)):
                if self.sections[i] == next_car.section():
                    break
                space += self.sections[i].length
            
            acceleration = 2*(space - self.speed*FOLLOWING_TIME) / FOLLOWING_TIME**2
            if acceleration < self.acceleration:
                self.acceleration = acceleration

    def update_speed(self):
        self.speed = max(0, min(MAX_SPEED, self.speed + self.acceleration))

    def update_distance(self):
        self.distance += self.speed
    
    def update_section(self):
        section = self.section()

        if self.distance > section.length:
            self.distance -= section.length

            if section.last_car == self:
                section.last_car = None
            if self.i == len(self.sections) - 1:
                self.done = True
            else:
                self.i += 1
                self.section().last_car = self
                
    def find_next_car(self, n=1):
        next_car = self.next_car
        
        if not next_car or next_car.done or next_car.section() not in self.sections:
            self.next_car = None
            
            for section in self.sections[self.i+n:]:
                self.next_car = section.last_car
                if self.next_car:
                    return

    def update(self):
        self.update_acceleration()
        self.update_speed()
        self.update_distance()
        self.update_section()
        self.find_next_car()
        

In [4]:
class Section():
    def __init__(self, x, y, a, b, length):
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        self.length = length
        self.last_car = None

In [5]:
class Line(Section):
    def position(self, s):
        return (self.x + self.a*s, self.y + self.b*s)

    def angle(self, s):
        return np.arctan2(-self.b, self.a)

In [6]:
class Curve(Section):
    def __init__(self, x, y, a, b, r):
        super().__init__(x, y, a, b, np.pi*r/2)
        self.r = r

    def position(self, s):
        c = self.a*np.pi/2 + self.b*s/self.r
        return (self.x+self.r*np.cos(c), self.y-self.r*np.sin(c))
    
    def angle(self, s):
        return (self.a+1)*np.pi/2 + self.b*s/self.r

In [7]:
class Intersection():
    def __init__(self, paths):
        self.cars = []
        self.paths = paths
        
    def spawn_cars(self):
        if random.random() < SPAWN_PROBABILITY:
            if random.random() < TRUCK_PROBABILITY:
                length = random.randint(*TRUCK_LENGTH)
            else:
                length = random.randint(*CAR_LENGTH)

            path = random.choice(self.paths)
            section0 = path.sections[0]
            
            if not section0.last_car or section0.last_car.distance > 80: 
                car = Car(length, path)
                self.cars.append(car)
                section0.last_car = car

    def update_cars(self):
        for car in self.cars:
            car.update()

            if car.done:
                self.cars.remove(car)
                
    def update(self):
        self.spawn_cars()
        self.update_cars()

In [8]:
class Path():
    def __init__(self, sections, blockades):
        self.sections = sections
        self.blockades = blockades

In [9]:
class Graphics():
    def __init__(self):
        self.screen = pygame.display.set_mode((H,H))

    def center(self, points):
        return np.array(points) + H/2
    
    def draw_car(self, car):
        p, l, w, a = car.position(), car.length, CAR_WIDTH, car.angle()
        corners = [[-l/2,w/2], [l/2,w/2], [l/2,-w/2], [-l/2,-w/2]]

        for c in corners:
            temp = c[0]*np.cos(a) - c[1]*np.sin(a) + p[0]
            c[1] = -c[0]*np.sin(a) - c[1]*np.cos(a) + p[1]
            c[0] = temp

        pygame.draw.polygon(self.screen, 1, self.center(corners))
    
    def draw(self, intersection):
        self.screen.fill((255, 255, 255))
        for car in intersection.cars:
            self.draw_car(car)
        pygame.display.flip()

In [10]:
class Simulation():
    def __init__(self, intersection, graphics):
        pygame.init()
        self.clock = pygame.time.Clock()
        self.intersection = intersection
        self.g = Graphics()
    
    def run(self):
        while True:
            self.clock.tick()
            self.intersection.update()
            self.g.draw(self.intersection)
    
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    return

In [11]:
L = 50

sections = [Line(L/2, H/2, 0, -1, H/2-L-B),
            Line(L/2, L+B, 0, -1, B),
            Line(L/2, L, 0, -1, L),
            Line(L/2, 0, 0, -1, L),
            Line(L/2, -L, 0, -1, H/2-L),
            Curve(L, L, 2, -1, L/2),
            Curve(0, 0, 0, 1, L/2)]

for _ in range(3):
    for s in sections[-7:]:
        if isinstance(s, Line):
            sections.append(Line(s.y,-s.x,s.b,-s.a,s.length))
        else:
            sections.append(Curve(s.y,-s.x,s.a+1,s.b,s.r))

l = [([0,1,2,3,4],[8]), ([0,1,5,25],[]), ([0,1,2,6,10,11],[8,15])]
paths = []

for i in range(4):
    for p, b in l:
        path_sections = [sections[(n+i*7)%28] for n in p]
        blockades = [sections[(n+i*7)%28] for n in b]
        paths.append(Path(path_sections, blockades))

        
intersection = Intersection(paths)
graphics = Graphics()
simulation = Simulation(intersection, graphics)
simulation.run()